In [30]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math
from IPython.core.magic import register_line_magic
from IPython.display import Javascript

Load Dataset

In [31]:
file_path = '/content/Busaha_Dataset.csv'
try:
  df = pd.read_csv(file_path)
  print("File Loaded!")
except:
  !gdown --id 1n3pwrW7E7Em-9F19BPQ6siZ4CuRsKS82
  try:
    df = pd.read_csv(file_path)
    print("File Loaded!")
  except:
    print("Something's wrong! check your path!!!")

File Loaded!


Checking Dataset

In [32]:
df.head()

,Unnamed: 0,jenis_usaha,Modal,modal_minimum,modal_maximum,tingkat_persaingan,tingkat_persaingan_idx,jumlah_pegawai,pegawai_minimum,omset,...,sosial_media,sosial_media_idx,barang_toserba,toko_toserba,toko_toserba_idx,tipe_toko,tipe_toko_idx,usaha_lain,tipe_usaha_lain,tipe_usaha_lain_idx
0,0,Transportasi,> 25jt,25000000,100000000,Sedang,1,2-4 orang,2,> 50 Juta,...,no_info,0,no_info,no_info,0,no_info,0,jasa sewa kendaraan,EX,1
1,1,Perternakan,10 - 25jt,10000000,25000000,Sedang,1,1 orang (Anda sendiri),1,10-50 Juta,...,no_info,0,no_info,no_info,0,no_info,0,no_info,no_info,0
2,2,Toserba/Toko Kelontong,1jt - 10 jt,1000000,10000000,Sedang,1,1 orang (Anda sendiri),1,< 10 Juta,...,no_info,0,Sembako,TRUE,1,Ruko,1,no_info,no_info,0
3,3,Kuliner,1jt - 10 jt,1000000,10000000,Sedang,1,1 orang (Anda sendiri),1,< 10 Juta,...,no_info,0,no_info,no_info,0,no_info,0,no_info,no_info,0
4,4,Dropshipper,1jt - 10 jt,1000000,10000000,Rendah,0,1 orang (Anda sendiri),1,< 10 Juta,...,WA,1,no_info,no_info,0,no_info,0,no_info,no_info,0


The dataset doesn't look very good to be a model, so it needs to be preprocessed before being used as a model

In [33]:
# Deleting unused row
delrow = ["Transportasi", "Toko Buah", "Penggilingan", "Alat Kecantikan", "Koleksi",
          "Alat kesehatan", "Perseroan Komanditer", "Jasa Isi Ulang Air Minum", "Jasa Mengajar", "Fotocopy"
          ]
for row in delrow:
  df.drop(df[df['jenis_usaha'] == row].index, inplace = True)

In [34]:
# Deleting unused column
try:
  df = df.drop(columns=['Unnamed: 0','Modal','tingkat_persaingan','jumlah_pegawai','omset',
                      'target_pasar','lokasi','hewan_ternak','hewan_ternak_sub','lokasi_geografis',
                      'mengerti_hewan','luas_lahan','usaha_kuliner','tipe_usaha_kuliner','kemampuan_masak',
                      'barang_dropship','penyimpanan_barang','biaya_internet','sosial_media','barang_toserba',
                      'toko_toserba','tipe_toko','usaha_lain','tipe_usaha_lain', 'modal_minimum', 'omset_minimum',
                      'hewan_ternak_idx', 'lokasi_geografis_idx', 'tipe_usaha_kuliner_idx'])
  print("Columns Dropped!")
except:
  print("Columns Already Dropped!")

Columns Dropped!


In [35]:
df.head()

,jenis_usaha,modal_maximum,tingkat_persaingan_idx,pegawai_minimum,tp_b,tp_m,tp_a,lokasi_idx,mengerti_hewan_idx,lahan_minimum,kemampuan_masak_idx,penyimpanan_barang_idx,internet_minimum,sosial_media_idx,toko_toserba_idx,tipe_toko_idx,tipe_usaha_lain_idx
1,Perternakan,25000000,1,1,1,1,1,1,1,1000,0,0,0,0,0,0,0
2,Toserba/Toko Kelontong,10000000,1,1,1,1,0,1,0,0,0,0,0,0,1,1,0
3,Kuliner,10000000,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0
4,Dropshipper,10000000,0,1,1,0,0,0,0,0,0,1,100000,1,0,0,0
5,Dropshipper,10000000,2,1,1,1,0,0,0,0,0,1,300000,1,0,0,0


Indexing Label

In [36]:
label = "jenis_usaha"
classes = [0]

In [37]:
if type(classes[0]) is int:
  classes = df[label].unique().tolist()
  df[label] = df[label].map(classes.index)
  print(f"Label classes: {classes}")
else:
  print("Label classes Already indexed!")
  print(f"Label classes: {classes}")

Label classes: ['Perternakan', 'Toserba/Toko Kelontong', 'Kuliner', 'Dropshipper', 'Barang Digital', 'Elektronik', 'Pakaian', 'Kosmetik']


In [39]:
df.head()

,jenis_usaha,modal_maximum,tingkat_persaingan_idx,pegawai_minimum,tp_b,tp_m,tp_a,lokasi_idx,mengerti_hewan_idx,lahan_minimum,kemampuan_masak_idx,penyimpanan_barang_idx,internet_minimum,sosial_media_idx,toko_toserba_idx,tipe_toko_idx,tipe_usaha_lain_idx
1,0,25000000,1,1,1,1,1,1,1,1000,0,0,0,0,0,0,0
2,1,10000000,1,1,1,1,0,1,0,0,0,0,0,0,1,1,0
3,2,10000000,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0
4,3,10000000,0,1,1,0,0,0,0,0,0,1,100000,1,0,0,0
5,3,10000000,2,1,1,1,0,0,0,0,0,1,300000,1,0,0,0


In [41]:
# # Randomize data
# random = df.sample(frac=1, random_state=30)
# # separating labels and features
# X = random.drop("jenis_usaha", axis=1).values
# y = random.jenis_usaha.values

In [42]:
# separating labels and features without randomize
X = df.drop("jenis_usaha", axis=1).values
y = df.jenis_usaha.values

In [43]:
# normalization
from sklearn.preprocessing import normalize
X_normalized = normalize(X, axis = 0, norm='l2')

In [44]:
# splitting data
total_length=len(df)
train_length=int(0.7*total_length)
test_length=int(0.3*total_length)

X_train=X_normalized[:train_length]
X_test=X_normalized[train_length:]
y_train=y[:train_length]
y_test=y[train_length:]

print("Length of train set x:",X_train.shape[0],"y:",y_train.shape[0])
print("Length of test set x:",X_test.shape[0],"y:",y_test.shape[0])

Length of train set x: 86 y: 86
Length of test set x: 37 y: 37


In [45]:
from keras.utils import np_utils
y_train=np_utils.to_categorical(y_train,num_classes=len(classes))
y_test=np_utils.to_categorical(y_test,num_classes=len(classes))
print("Shape of y_train",y_train.shape)
print("Shape of y_test",y_test.shape)

Shape of y_train (86, 8)
Shape of y_test (37, 8)


In [46]:
from keras.models import Sequential 
from keras.layers import Dense,Activation,Dropout 

shape = X_train.shape[1]

model=Sequential()
model.add(Dense(128,input_dim=X_train.shape[1],activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(classes),activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [47]:
class cb(tf.keras.callbacks.Callback): 
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.90 and logs.get('val_loss') < 0.4 and logs.get('loss') < 0.4 and logs.get('val_accuracy') > 0.80):
      self.model.stop_training = True

callbacks = cb()

In [48]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=20,epochs=500,verbose=1, callbacks=[callbacks])

Epoch 1/500
5/5 [==============================] - 1s 86ms/step - loss: 2.0633 - accuracy: 0.2442 - val_loss: 2.0285 - val_accuracy: 0.5676
Epoch 2/500
5/5 [==============================] - 0s 8ms/step - loss: 2.0286 - accuracy: 0.4070 - val_loss: 1.9957 - val_accuracy: 0.1622
Epoch 3/500
5/5 [==============================] - 0s 8ms/step - loss: 1.9899 - accuracy: 0.3953 - val_loss: 1.9535 - val_accuracy: 0.1892
Epoch 4/500
5/5 [==============================] - 0s 7ms/step - loss: 1.9519 - accuracy: 0.4070 - val_loss: 1.9108 - val_accuracy: 0.1351
Epoch 5/500
5/5 [==============================] - 0s 7ms/step - loss: 1.9110 - accuracy: 0.3837 - val_loss: 1.8596 - val_accuracy: 0.1351
Epoch 6/500
5/5 [==============================] - 0s 7ms/step - loss: 1.8638 - accuracy: 0.3953 - val_loss: 1.8047 - val_accuracy: 0.1622
Epoch 7/500
5/5 [==============================] - 0s 7ms/step - loss: 1.8174 - accuracy: 0.4186 - val_loss: 1.7482 - val_accuracy: 0.1622
Epoch 8/500
5/5 [=========

In [49]:
peternak = [25000000, 1, 1, 1, 1, 1, 1, 1, 1000, 0, 0, 0, 0, 0, 0, 0]
toserba = [10000000, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]
kuliner = [25000000, 1, 2, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]

In [50]:
fitur_normalized = normalize([peternak], axis = 0)
prediction = model.predict([fitur_normalized])

clas = np.argmax(prediction, axis = 1)
clas1 = np.argsort(clas)
print(clas)
indextion = np.argsort((-prediction), axis = 1)[0,:3]
top1, top2, top3 = indextion
indextion0 = np.argsort((-prediction), axis = 1)[0]
indextion1 = np.argmax(prediction, axis = 1)
indextion2 = np.argmin(prediction)
print(f"top 3 prediksi adalah : {indextion}")
print(f"keseluruhan prediksi adalah : {indextion0}")
print(f"prediksi paling mendekati adalah : {indextion1}")
print(f"prediksi paling tidak mendekati adalah : {indextion2}")
print(f"label top 3 prediksi adalah : {classes[top1]}, {classes[top2]}, {classes[top3]}")

[0]
top 3 prediksi adalah : [0 5 1]
keseluruhan prediksi adalah : [0 5 1 6 4 7 2 3]
prediksi paling mendekati adalah : [0]
prediksi paling tidak mendekati adalah : 3
label top 3 prediksi adalah : Perternakan, Elektronik, Toserba/Toko Kelontong


In [51]:
fitur_normalized = normalize([toserba], axis = 0)
prediction = model.predict([fitur_normalized])

clas = np.argmax(prediction, axis = 1)
clas1 = np.argsort(clas)
print(clas)
indextion = np.argsort((-prediction), axis = 1)[0,:3]
top1, top2, top3 = indextion
indextion0 = np.argsort((-prediction), axis = 1)[0]
indextion1 = np.argmax(prediction, axis = 1)
indextion2 = np.argmin(prediction)
print(f"top 3 prediksi adalah : {indextion}")
print(f"keseluruhan prediksi adalah : {indextion0}")
print(f"prediksi paling mendekati adalah : {indextion1}")
print(f"prediksi paling tidak mendekati adalah : {indextion2}")
print(f"label top 3 prediksi adalah : {classes[top1]}, {classes[top2]}, {classes[top3]}")

[1]
top 3 prediksi adalah : [1 6 0]
keseluruhan prediksi adalah : [1 6 0 2 7 5 3 4]
prediksi paling mendekati adalah : [1]
prediksi paling tidak mendekati adalah : 4
label top 3 prediksi adalah : Toserba/Toko Kelontong, Pakaian, Perternakan


In [52]:
fitur_normalized = normalize([kuliner], axis = 0)
prediction = model.predict([fitur_normalized])

clas = np.argmax(prediction, axis = 1)
clas1 = np.argsort(clas)
print(clas)
indextion = np.argsort((-prediction), axis = 1)[0,:3]
top1, top2, top3 = indextion
indextion0 = np.argsort((-prediction), axis = 1)[0]
indextion1 = np.argmax(prediction, axis = 1)
indextion2 = np.argmin(prediction)
print(f"top 3 prediksi adalah : {indextion}")
print(f"keseluruhan prediksi adalah : {indextion0}")
print(f"prediksi paling mendekati adalah : {indextion1}")
print(f"prediksi paling tidak mendekati adalah : {indextion2}")
print(f"label top 3 prediksi adalah : {classes[top1]}, {classes[top2]}, {classes[top3]}")

[2]
top 3 prediksi adalah : [2 1 6]
keseluruhan prediksi adalah : [2 1 6 7 0 5 3 4]
prediksi paling mendekati adalah : [2]
prediksi paling tidak mendekati adalah : 4
label top 3 prediksi adalah : Kuliner, Toserba/Toko Kelontong, Pakaian


In [53]:
model.save("/content/model.h5")

In [54]:
new_model = tf.keras.models.load_model('/content/model.h5')

In [55]:
fitur_normalized = normalize([kuliner], axis = 0)
pred = new_model.predict([fitur_normalized])
np.argmax(pred, axis=1)

array([2])

In [56]:
fitur_normalized = normalize([peternak], axis = 0)
pred = new_model.predict([fitur_normalized])
np.argmax(pred, axis=1)

array([0])

In [57]:
new_model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 7ms/step - loss: 0.0033 - accuracy: 1.0000


[0.003302990924566984, 1.0]

In [58]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model_busaha.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmppamdxvhe/assets


INFO:tensorflow:Assets written to: /tmp/tmppamdxvhe/assets
